In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install pyarrow


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import zipfile 
import io
import os
import concurrent.futures
import time
import subprocess
import multiprocessing
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow
import json
from matplotlib.ticker import FormatStrFormatter

In [5]:
import sys
import os

sys.path.append(os.path.abspath('../advmla_at3_package'))

In [6]:
df_train = pd.read_feather('../data/processed/train_data.feather')
df_test = pd.read_feather('../data/processed/test_data.feather')

In [7]:
df_train.shape

(5106602, 13)

In [8]:
df_train.head(10)

,searchDate,flightDate,startingAirport,destinationAirport,isNonStop,totalFare,totalTravelDistance,segmentsArrivalAirportCode,DepartureTimeHour,CabinCode,AirlineNameScore,date_diff_days,weekday
3767896,2022-04-18,2022-05-06,LGA,DFW,True,93.599998,1380.0,DFW,8,1.0,2,18,4
929418,2022-05-11,2022-05-18,BOS,IAD,False,207.600006,406.0,JFK||IAD,11,1.0,4,7,2
5456598,2022-05-08,2022-06-21,ORD,EWR,False,358.600006,725.0,DTW||EWR,14,1.0,4,44,1
4876708,2022-05-08,2022-05-18,OAK,BOS,False,728.599976,2688.0,SLC||BOS,17,1.0,4,10,2
2004101,2022-05-12,2022-07-01,DFW,ATL,True,228.600006,725.0,ATL,19,1.0,4,50,4
1057863,2022-05-09,2022-06-29,BOS,PHL,True,93.599998,280.0,PHL,11,1.0,4,51,2
96461,2022-05-07,2022-05-17,ATL,EWR,True,213.600006,762.0,EWR,17,1.0,4,10,1
5565752,2022-05-02,2022-06-09,PHL,ORD,False,339.579987,NaN,MCO||ORD,7,1.0,1,38,3
4455243,2022-05-09,2022-06-15,MIA,LGA,True,140.600006,1104.0,LGA,12,1.0,4,37,2
3553263,2022-05-05,2022-05-15,LAX,PHL,False,821.590027,3335.0,FLL||PHL,10,1.0,2,10,6


In [9]:
df_train.isnull().sum()

searchDate                         0
flightDate                         0
startingAirport                    0
destinationAirport                 0
isNonStop                          0
totalFare                          0
totalTravelDistance           484110
segmentsArrivalAirportCode         0
DepartureTimeHour                  0
CabinCode                          0
AirlineNameScore                   0
date_diff_days                     0
weekday                            0
dtype: int64

In [10]:
df_test.isnull().sum()

searchDate                         0
flightDate                         0
startingAirport                    0
destinationAirport                 0
isNonStop                          0
totalFare                          0
totalTravelDistance           120986
segmentsArrivalAirportCode         0
DepartureTimeHour                  0
CabinCode                          0
AirlineNameScore                   0
date_diff_days                     0
weekday                            0
dtype: int64

In [13]:
from engineering.clean_master import preprocessing_data

df_train_processed, df_test_precessed = preprocessing_data(df_train, df_test)

In [14]:
selected_features = ['date_diff_days', 'weekday', 'CabinCode', 'DepartureTimeHour', 'totalTravelDistance']
categorical_features = ['startingAirport', 'destinationAirport', 'isNonStop']
target = 'totalFare'

df_filtered_train = df_train_processed[categorical_features + selected_features + [target]]
df_filtered_test = df_test_precessed[categorical_features + selected_features + [target]]

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LinearRegression

numerical_features = ['totalTravelDistance']

preprocessor = ColumnTransformer(
    transformers=[
        ('one-hot', OneHotEncoder(handle_unknown='ignore'), categorical_features),  
        ('scaler', StandardScaler(), numerical_features) 
    ],
    remainder='passthrough')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression()) 
])

In [20]:
from sklearn.model_selection import train_test_split


# Split the datasets
train_df, val_df = train_test_split(df_filtered_train, test_size=0.2, random_state=42)

X_train = train_df.drop(columns=[target], axis=1)  
y_train = train_df[target]

X_val = val_df.drop(columns=[target], axis=1)  
y_val = val_df[target]

X_test = df_filtered_test.drop(columns=[target], axis=1)  
y_test = df_filtered_test[target]

In [17]:
from modeling.null import NullRegressor

base_model = NullRegressor()

y_base = base_model.fit_predict(y_train)


In [18]:
# Baseline result

from modeling.performance import print_regressor_scores

print_regressor_scores(y_preds=y_base, y_actuals=y_train, set_name='Training')
print_regressor_scores(y_preds=base_model.predict(y_val), y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_preds=base_model.predict(y_test), y_actuals=y_test, set_name='Testing')

RMSE Training: 190.866455078125
MAE Training: 136.70587158203125
RMSE Validation: 189.75318908691406
MAE Validation: 136.62557983398438
RMSE Testing: 191.07180786132812
MAE Testing: 136.75062561035156


In [21]:
pipeline.fit(X_train, y_train)

y_preds = pipeline.predict(X_train)
y_val_preds = pipeline.predict(X_val)
y_test_preds = pipeline.predict(X_test)

In [22]:
print_regressor_scores(y_preds, y_train, set_name='Training')
print_regressor_scores(y_val_preds, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds, y_actuals=y_test, set_name='Testing')

RMSE Training: 136.07393438739464
MAE Training: 96.59250108703283
RMSE Validation: 135.21356822677274
MAE Validation: 96.54232007646003
RMSE Testing: 135.8049850296426
MAE Testing: 96.63959049758105
